In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Importing libraries for portfolio optimization
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [3]:
from scipy.optimize import minimize

In [4]:
from distfit import distfit

In [5]:
!python -V

Python 3.10.13


In [6]:
%pip list

Package                      Version        Editable project location
---------------------------- -------------- ---------------------------
absl-py                      2.1.0
accelerate                   0.26.0
aiohappyeyeballs             2.4.3
aiohttp                      3.11.8
aiosignal                    1.3.1
arrow                        1.3.0
asttokens                    2.4.1
astunparse                   1.6.3
async-timeout                5.0.1
attrs                        24.2.0
beautifulsoup4               4.11.1
blis                         0.7.11
boto3                        1.35.71
botocore                     1.35.71
cachetools                   5.5.0
catalogue                    2.0.10
certifi                      2024.8.30
cffi                         1.17.1
charset-normalizer           3.4.0
clarabel                     0.9.0
click                        7.1.2
colorama                     0.4.6
coloredlogs                  15.0.1
colourmap                    1.1.19
c

### investments tickers

In [7]:
symbol_type = "ETF"

In [8]:
yf_market_info  = pd.read_csv('../data/market_info.csv')

In [9]:
ticker_info_df = yf_market_info[yf_market_info.symbol_type==symbol_type].reset_index()
ticker_list = list(ticker_info_df.symbol)
ticker_name = list(ticker_info_df.symbol_name)
ticker_currency = list(ticker_info_df.unit)
ticker_dict = dict(map(lambda i,j : (i,j) , ticker_list,ticker_name))
ticker_currency_dict = dict(map(lambda i,j : (i,j) , ticker_list,ticker_currency))

In [10]:
ticker_dict

{'SPXP.L': 'SP500',
 'FTAL.L': 'FTSEALL',
 'CNKY.L': 'NIKKEI225',
 'S600.L': 'STOXX600',
 'C300.L': 'CHINA300',
 'HMAF.L': 'ASIA',
 'FLXI.L': 'INDIA',
 'FVUB.L': 'BRAZIL',
 'USDGBP=X': 'US dollar ex rate'}

In [11]:
ticker_currency_dict

{'SPXP.L': 'GBP',
 'FTAL.L': 'GBP',
 'CNKY.L': 'GBP',
 'S600.L': 'GBP',
 'C300.L': 'USD',
 'HMAF.L': 'GBP',
 'FLXI.L': 'USD',
 'FVUB.L': 'GBP',
 'USDGBP=X': 'USD'}

In [12]:
ticker_symbols  = " ".join(ticker_list)
ticker_symbols

'SPXP.L FTAL.L CNKY.L S600.L C300.L HMAF.L FLXI.L FVUB.L USDGBP=X'

In [13]:
#ticker_symbols = ticker_symbols + " EURGBP=X"

In [14]:
data = yf.download(ticker_symbols, period='1y')
data.head(5)

[*********************100%***********************]  9 of 9 completed


Price        Close                                                       \
Ticker      C300.L   CNKY.L     FLXI.L     FTAL.L     FVUB.L     HMAF.L   
Date                                                                      
2024-01-22  3.9805  19376.0  40.587502  60.610001  20.785000  32.314999   
2024-01-23  4.0370  19237.0  39.430000  60.575001  20.915001  32.939999   
2024-01-24  4.1520  19220.0  40.132500  60.930000  21.209999  33.455002   
2024-01-25  4.1165  19141.0  40.014999  60.924999  21.270000  33.430000   
2024-01-26  4.1355  19041.0  40.064999  61.724998  21.325001  33.419998   

Price                                   High  ...     Open Volume           \
Ticker      S600.L   SPXP.L USDGBP=X  C300.L  ... USDGBP=X C300.L   CNKY.L   
Date                                          ...                            
2024-01-22  9602.0  74346.5  0.78741  3.9805  ...  0.78735    0.0  18244.0   
2024-01-23  9566.0  74488.5  0.78697  4.0270  ...  0.78689  300.0    563.0   
2024-01-24  9678.0  74769.5  0.78790  4.1520  ...  0.78776    0.0   3701.0   
2024-01-25  9676.5  75027.5  0.78624  4.1070  ...  0.78626    1.0    204.0   
2024-01-26  9801.0  75044.0  0.78689  4.1360  ...  0.78697  182.0   2445.0   

Price                                                                 
Ticker       FLXI.L   FTAL.L  FVUB.L  HMAF.L S600.L  SPXP.L USDGBP=X  
Date                                                                  
2024-01-22  21504.0   5082.0  1355.0   111.0  317.0   979.0        0  
2024-01-23  20304.0   5069.0  2820.0  7922.0  570.0  1662.0        0  
2024-01-24  26709.0  15558.0    94.0  4071.0   41.0   818.0        0  
2024-01-25  16145.0   5899.0   127.0  2113.0  951.0  1011.0        0  
2024-01-26  30807.0   2642.0   761.0  1540.0    0.0   897.0        0  

[5 rows x 45 columns]

In [15]:
# forward fill exchange rate
#data['USDGBP=X'] = data['USDGBP=X'].ffill()

### returns

In [16]:
# some funds have just 5 year history - so ok for upto 2 years
yf_period = "1y" # ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
yf_interval = "1d"

In [17]:
#  1.  Create dataframe yf_price with yf.download
yf_df = yf.download(
        tickers = ticker_symbols,       # tickers list or string as well
        period = yf_period,      # optional, default is '1mo'
        interval = yf_interval,  # fetch data by intervaal
        group_by = 'ticker',     # group by ticker
        auto_adjust = True,      # adjust all OHLC (open-high-low-close)
        prepost = True,          # download market hours data
        threads = True,          # threads for mass downloading
        proxy = None)            # proxy

#  2.  Select 'Close' (price at market close) column only
yf_df = yf_df.iloc[:, yf_df.columns.get_level_values(1)=='Close']

#  3.  Remove the dataframe multi-index
yf_df.columns = yf_df.columns.droplevel(1)  # multi-index

[*********************100%***********************]  9 of 9 completed


### forward fill exchange rate

In [18]:
# forward fill exchange rate
yf_df['USDGBP=X'] = yf_df['USDGBP=X'].ffill()

### convert USD ETF's to GBP

In [19]:
for i in ticker_currency_dict:
    if ticker_currency_dict[i]== 'USD':
        # convert column
        yf_df[i] = yf_df[i].multiply(yf_df["USDGBP=X"], axis="index")

In [20]:
yf_name_df = yf_df.rename(columns=ticker_dict)

In [21]:
yf_name_df.isnull().sum()

Ticker
US dollar ex rate    0
CHINA300             8
SP500                7
NIKKEI225            7
BRAZIL               7
ASIA                 7
INDIA                7
FTSEALL              7
STOXX600             7
dtype: int64

In [22]:
yf_name_df.dropna(inplace=True)

In [28]:
yf_name_df.shape

(252, 9)

In [29]:
yf_name_df.drop(columns="US dollar ex rate", axis=0, inplace=True)

In [30]:
yf_name_df

Ticker,CHINA300,SP500,NIKKEI225,BRAZIL,ASIA,INDIA,FTSEALL,STOXX600
Date,,,,,,,,
2024-01-22,3.134286,74346.5,19376.0,20.785000,32.314999,31.959005,60.610001,9602.0
2024-01-23,3.176998,74488.5,19237.0,20.915001,32.939999,31.030228,60.575001,9566.0
2024-01-24,3.271361,74769.5,19220.0,21.209999,33.455002,31.620395,60.930000,9678.0
2024-01-25,3.236557,75027.5,19141.0,21.270000,33.430000,31.461392,60.924999,9676.5
2024-01-26,3.254183,75044.0,19041.0,21.325001,33.419998,31.526746,61.724998,9801.0
...,...,...,...,...,...,...,...,...
2025-01-13,3.790471,93904.5,20147.0,17.059000,39.470001,34.221785,68.160004,10393.0
2025-01-14,3.881818,94251.0,20118.0,17.035000,39.924999,34.565449,67.980003,10436.0
2025-01-15,3.876671,95526.5,20274.5,17.212999,40.215000,34.863228,68.949997,10554.0


### returns

In [35]:
#Create a new columns Return in the dataset for each company's
yf_name_df['R_CHINA300'] = yf_name_df[['CHINA300']].pct_change(1)
yf_name_df['R_SP500'] = yf_name_df[['SP500']].pct_change(1)
yf_name_df['R_NIKKEI225'] = yf_name_df[['NIKKEI225']].pct_change(1)
yf_name_df['R_BRAZIL'] = yf_name_df[['BRAZIL']].pct_change(1)
yf_name_df['R_ASIA'] = yf_name_df[['ASIA']].pct_change(1)
yf_name_df['R_FTSEALL'] = yf_name_df[['FTSEALL']].pct_change(1)
yf_name_df['R_STOXX600'] = yf_name_df[['STOXX600']].pct_change(1)

In [36]:
returns  = yf_name_df[['R_CHINA300','R_SP500','R_NIKKEI225','R_BRAZIL','R_ASIA', 'R_FTSEALL','R_STOXX600']]

### sharpe ratio

In [37]:
# The Sharpe ratio gives the excess return of a stock (or a portfolio) per unit of risk.

In [38]:
# set the number of days in this sample to be the number of rows
n_days = returns.shape[0]
def sharpe_ratio (serie, rf = 0.045):
    ''''
    serie : Series of daily returns of the stock or portfolio
    rf: annual return on a risk-free investment (default rf=0.045)
    '''''
    # Mean Annualized Return
    rp = serie.mean()*n_days
    # Annual volatility
    volatility = serie.std()*np.sqrt(n_days)
    # Sharpe ratio
    sharpe = (rp - rf) / volatility
    return sharpe

In [39]:
for col in returns.columns:
    print("Sharpe ratio:", col,"=", round(sharpe_ratio(yf_name_df[col]),2 ))

Sharpe ratio: R_CHINA300 = 0.74
Sharpe ratio: R_SP500 = 2.01
Sharpe ratio: R_NIKKEI225 = 0.15
Sharpe ratio: R_BRAZIL = -0.97
Sharpe ratio: R_ASIA = 1.28
Sharpe ratio: R_FTSEALL = 1.16
Sharpe ratio: R_STOXX600 = 0.73


### portfolio optimisation

In [26]:
# Portfolio Simulation
portfolio        = 2673  # generation of a portfolio
n_assets         = returns.shape[1]
weights          = np.random.dirichlet(np.full(n_assets,0.05),portfolio)
mean_returns     = returns.mean()
sigma            = returns.cov()
expected_returns = np.zeros(portfolio)
expected_vol     = np.zeros(portfolio)
sharpe_ratio     = np.zeros(portfolio)
rf_rate          = 0.0                          # risk free rate

for i in range(portfolio):
    w                   = weights[i,:]
    expected_returns[i] = np.sum(mean_returns @ w)*252
    expected_vol[i]     = np.sqrt(np.dot(w.T,sigma @ w))*np.sqrt(252)
    sharpe_ratio[i]     = (expected_returns[i]-rf_rate)/expected_vol[i]

NameError: name 'returns' is not defined

In [ ]:
def portfolio_volatility(weight):
    return np.sqrt(np.dot(weight.T,
                          np.dot(sigma,weight)))*np.sqrt(252)

def portfolio_return(weight):

    return np.sum(mean_returns*weight)*252

def portfolio_performance(weight):
    return_p = portfolio_return(weight)
    vol_p    = portfolio_volatility(weight)
    return return_p, vol_p

In [ ]:
def negativeSR(weight):
    return_p, vol_p = portfolio_performance(weight)
    rf_rate         = 0.025
    return -(return_p - rf_rate)/vol_p

def max_sharpe_ratio():

    def sum_one(weight):
        w= weight
        return np.sum(weight)-1

    n_assets           = returns.shape[1]
    weight_constraints = ({'type':'eq','fun': sum_one})
    w0                 = np.random.dirichlet(np.full(n_assets,0.05)).tolist()    # w0 is an initila guess

    return minimize(negativeSR,w0,method='SLSQP',
                          bounds  =((0,1),)*n_assets,
                      constraints = weight_constraints)

In [ ]:
def min_vol():

    n_assets           = returns.shape[1]
    weight_constraints = ({'type':'eq','fun': lambda x: np.sum(x)-1})
    w0                 = np.random.dirichlet(np.full(n_assets,0.05)).tolist()
    bounds             = ((0,1),)*n_assets

    return minimize(portfolio_volatility,w0,method='SLSQP',
                   bounds      = bounds,
                   constraints = weight_constraints)

In [ ]:
def efficient_portfolio_target(target):

    constraints = ({'type':'eq','fun': lambda x: portfolio_return(x)- target},
                  {'type' :'eq','fun': lambda x: np.sum(x)-1})
    w0          = np.random.dirichlet(np.full(n_assets,0.05)).tolist()
    bounds      = ((0,1),)*n_assets

    return minimize(portfolio_volatility,w0, method = 'SLSQP',
                    bounds      = bounds,
                    constraints = constraints)

In [ ]:
def efficient_frontier(return_range):
    return [efficient_portfolio_target(ret) for ret in return_range]

In [ ]:
sharpe_maximum      = max_sharpe_ratio()
return_p,vol_p      = portfolio_performance(sharpe_maximum['x'])
min_volatility      = min_vol()
return_min,vol_min  = portfolio_performance(min_volatility['x'])


plt.figure(figsize =(15,10))
plt.style.use('ggplot')
plt.scatter(expected_vol,expected_returns, c = sharpe_ratio)
# plt.colorbar.sel(label = 'Sharpe Ratio',size=20)
plt.colorbar().set_label('Sharpe Ratio', size= 20, color = 'g', family='serif',weight='bold')
target               = np.linspace(return_min,1.02,100)
efficient_portfolios = efficient_frontier(target)
plt.plot([i.fun for i in efficient_portfolios], target, linestyle ='dashdot', color ='black',
         label='Efficient Frontier')
plt.scatter(vol_p,return_p, c = 'r', marker='*', s = 500, label = 'Maximum Sharpe Ratio')
plt.scatter(vol_min,return_min, c = 'g',  marker ='*', s = 500, label='Minimum Volatility Portfolio')

font1 = {'family':'serif','color':'darkred','size':20,'weight':'bold'}
font2 = {'family':'serif','color':'darkred','size':20,'weight':'bold'}
plt.title('Portfolio Optimization based on Efficient Frontier',fontdict=font1)
plt.xlabel('Annualised Volatility',fontdict=font2)
plt.ylabel('Annualised Returns',fontdict=font2)
plt.legend(labelspacing=0.8)

### statistics

In [ ]:
daily_returns_df = yf_name_df.pct_change()

In [ ]:
daily_returns_df.mean(skipna=True)

In [ ]:
daily_returns_df.std(skipna=True)

In [ ]:
daily_returns_df.kurtosis()

In [ ]:
daily_returns_df.skew()

In [ ]:
# Pairplots
sns.pairplot(daily_returns_df, kind = 'reg')
plt.show()

In [ ]:
yf_name_df

In [ ]:
from scipy import stats
with sns.axes_style("whitegrid"):
    fig, axes = plt.subplots(nrows=9, ncols=2, figsize=(18,12))

    axes[0][0].plot(yf_name_df['FTSEALL'], color='blue')
    axes[0][0].set_title('Daily Returns FTSE all-share')

    sns.distplot(yf_name_df['FTSEALL'], norm_hist=True, fit=stats.norm, color='blue',
                bins=50, ax=axes[0][1])
    axes[0][1].set_title('Density of FTSE all-share')

    
    axes[1][0].plot(yf_name_df['STOXX600'], color='green')
    axes[1][0].set_title('Daily Returns of STOXX Europe 600')

    sns.distplot(yf_name_df['STOXX600'], norm_hist=True, fit=stats.norm, color='green',
                bins=50, ax=axes[1][1])
    axes[1][1].set_title('Density of STOXX Europe 600')

    
    axes[2][0].plot(yf_name_df['NIKKEI225'], color='brown')
    axes[2][0].set_title('Daily Returns NIKKEI 225')

    sns.distplot(yf_name_df['NIKKEI225'], norm_hist=True, fit=stats.norm, color='brown',
                bins=50, ax=axes[2][1])
    axes[2][1].set_title('Density of NIKKEI 225')

    
    axes[3][0].plot(yf_name_df['SP500'], color='red')
    axes[3][0].set_title('Daily Returns S&P 500')

    sns.distplot(yf_name_df['SP500'], norm_hist=True, fit=stats.norm, color='red',
                bins=50, ax=axes[3][1])
    axes[3][1].set_title('Density of S&P 500')

    axes[4][0].plot(yf_name_df['ASIA'], color='red')
    axes[4][0].set_title('Daily Returns Asia (ex Japan)')

    sns.distplot(yf_name_df['ASIA'], norm_hist=True, fit=stats.norm, color='red',
                bins=50, ax=axes[4][1])
    axes[4][1].set_title('Density of Asia (ex Japan)')

    
    axes[5][0].plot(yf_name_df['CHINA300'], color='red')
    axes[5][0].set_title('Daily Returns China 300')

    sns.distplot(yf_name_df['CHINA300'], norm_hist=True, fit=stats.norm, color='red',
                bins=50, ax=axes[5][1])
    axes[5][1].set_title('Density of China 300')

    
    axes[6][0].plot(yf_name_df['INDIA'], color='yellow')
    axes[6][0].set_title('Daily Returns India')

    sns.distplot(yf_name_df['INDIA'], norm_hist=True, fit=stats.norm, color='yellow',
                bins=50, ax=axes[6][1])
    axes[6][1].set_title('Density of India')

    axes[7][0].plot(yf_name_df['BRAZIL'], color='blue')
    axes[7][0].set_title('Daily Returns Brazil')

    sns.distplot(yf_name_df['BRAZIL'], norm_hist=True, fit=stats.norm, color='blue',
                bins=50, ax=axes[7][1])
    axes[7][1].set_title('Density of Brazil')

    axes[8][0].plot(yf_name_df['BITCOIN'], color='blue')
    axes[8][0].set_title('Daily Returns Bitcoin')

    sns.distplot(yf_name_df['BITCOIN'], norm_hist=True, fit=stats.norm, color='blue',
                bins=50, ax=axes[8][1])
    axes[8][1].set_title('Density of Bitcoin') 
    
    plt.tight_layout()
    plt.show()

### check for dividends - check all ETFs reinvest dividends i.e. accumulate (acc)

In [ ]:
ticker_list

In [ ]:
yf_dividend = pd.DataFrame()   # initialize dataframe

for i in ticker_list:
    x = pd.DataFrame(yf.Ticker(i).dividends)
    x = x.rename(columns={"Dividends":i})
    yf_dividend = pd.concat([yf_dividend,x], axis=1)
    if len(x) > 0:
        print('{:>8}\t- dividends'.format(i))
    else:
        print('{:>8}\t- no dividends'.format(i))             

In [ ]:
yf_dividend

### check currencies

In [ ]:
ticker_list

In [ ]:
tickers = yf.Tickers(ticker_symbols)
for i in ticker_list:
    print(f"{ticker_dict[i]} : {tickers.tickers[i].info['currency']}")

In [ ]:
tickers.tickers['FLXI.L'].info # india

In [ ]:
ticker_info_df

### correlation

### fit curves

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.hist(daily_returns_df,bins=100)
#plt.ylim(0,1500)
plt.xlim(-0.05,0.05)
plt.xlabel('daily_return %')
plt.ylabel('frequency')
plt.title('daily % returns 10yrs')

# Initialize distfit - ftse
dist = distfit()

# Determine best-fitting probability distribution for data
dist.fit_transform(daily_returns_df['FTAL.L'].dropna())

print(dist.summary)

# Plot results
dist.plot()
plt.show()

In [ ]:
# Correlation Matrix
corr = daily_returns_df.corr()
mask = np.zeros_like(corr, dtype=np.bool_)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corr, annot=True, mask = mask)
plt.show()

### portfolio - markowitz

In [ ]:
weights = [0.2, 0.2, 0.2, 0.2,0.2,0.2,0.2,0.2] # Defining weights for each stock
portfolio = yf_df.dot(weights) # Creating portfolio multiplying each stock for its respective weight 
portfolio # Displaying portfolio's daily returns

In [ ]:
# Calculating the annualized expected returns and the annualized sample covariance matrix
mu = expected_returns.mean_historical_return(yf_df) #expected returns
S = risk_models.sample_cov(yf_df) #Covariance matrix

In [ ]:
# Visualizing the annualized expected returns
mu

In [ ]:
# Visualizing the covariance matrix
S

In [ ]:
# Optimizing for maximal Sharpe ratio
ef = EfficientFrontier(mu, S) # Providing expected returns and covariance matrix as input
weights = ef.max_sharpe() # Optimizing weights for Sharpe ratio maximization 

clean_weights = ef.clean_weights() # clean_weights rounds the weights and clips near-zeros

# Printing optimized weights and expected performance for portfolio
clean_weights